In [1]:
import lyricsgenius # Genius lyrics provider API
from textblob import TextBlob # language detector
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import logging
import sys
import re
import numpy as np
import time
import tqdm

### Setup basic logging

In [2]:
logging.basicConfig(filename="/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/log_files/lyrics_scraping_reversed.log",
                   level=logging.INFO,
                   format='%(asctime)s %(levelname)s %(message)s')

### Functions for scraping artists and songs

In [3]:
def metrolyrics_retriever(output_df, 
                          driver, 
                          root_url="https://www.metrolyrics.com/top-artists.html", 
                          per_artist_limit=None, 
                          overall_limit=None):
    
    dataset = output_df
    alphabet, urls = retrieve_alphabet(root_url, driver)
    
    for letter, url in zip(alphabet, urls):
        
        artists = retrieve_artists(letter, url, driver)
        for artist, (artist_url, artist_genre) in artists.items():
            songs = retrieve_songs(artist, artist_url, driver, per_artist_limit=per_artist_limit)
            nans = [np.nan for i in range(len(songs))]
            genre = [artist_genre for i in range(len(songs))]
            
            artist_df = pd.DataFrame({"artist":[artist for i in range(len(songs))], 
                                      "song":songs, 
                                      "album":nans, 
                                      "release_date":nans, 
                                      "genre":genre, 
                                      "lyrics":nans})
            dataset = pd.concat([dataset, artist_df])
            dataset.reset_index(drop=True, inplace=True)
            
            
            print(f"\rSongs retrieved: {dataset.shape[0]}", end='\r')
            sys.stdout.flush()
            
            if not overall_limit is None:
                if dataset.shape[0] > overall_limit:
                    return dataset.iloc[:overall_limit, :]
        
        logging.info("Writing to csv")
        dataset.to_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/songs.csv", index=False)
        
    return dataset

def retrieve_alphabet(url, driver):
    
    driver.get(url)
    html = driver.page_source
    logging.info("Alphabet html source loaded")
    soup = BeautifulSoup(html, "lxml")
    
    try:
        alphabet = soup.find("p", class_="artist-letters").find_all("a")
        letters = []
        urls = []
        for idx, letter in enumerate(alphabet):
            try:
                letter_text = letter.get_text()
                letter_url = letter.get("href")
                letters.append(letter_text)
                urls.append(letter_url)
            except AttributeError:
                logging.warning(f"Unable to retrieve artists for letter number {idx}. Html structure not aligned")
    except AttributeError:
        logging.error("Unable to retrieve urls for letters. Html structure not aligned")
    
    logging.info(f"Got letters {letters}")
    
    return letters, urls
            
def retrieve_artists(letter, letter_url, driver):
    
    logging.info(f"\nRetrieving artists for letter {letter}")
    
    driver.get(letter_url)
    html = driver.page_source
    logging.info("Html source loaded")
    soup = BeautifulSoup(html, "lxml")
    
    artists = dict()
    
    # top six artists first
    logging.info("Retrieving top artists")
    try:
        top_artists = soup.find("div", class_="module top-artists clearfix")\
                            .find_all("div", class_=['artist grid_2 alpha', 'artist grid_2 ', "artist grid_2 omega"])
        for idx, artist in enumerate(top_artists):
            try:
                box = artist.find("a", class_="image")
                artist_url = box.get("href")
                artist_name = box.find("span", class_="name").get_text()
                artist_genre = ""
                substitute = re.compile("\s*(L|l)yrics.*")
                artist_name = re.sub(substitute, "", artist_name).replace("\n", "").replace("\t", "")
                artists[artist_name] = artist_url, artist_genre
                logging.info(f"Got {artist_name}")
            except AttributeError:
                logging.warning(f"Unable to retrieve top artist number {idx}. Html structure not aligned")        
    except AttributeError:
        logging.warning(f"Unable to retrieve top artists. Html structure not aligned")
    
    # other artists
    logging.info("Retrieving other artists")
    try:
        other_artists = soup.find("table", class_="songs-table").find("tbody").find_all("tr")
        for idx, artist in enumerate(other_artists):
            try:
                box = artist.find("a")
                artist_url = box.get("href")
                artist_name = box.get_text()
                artist_genre = artist.find_all("td")[1].get_text().replace("\n", "").replace("\t", "")
                if artist_genre == "":
                    artist_genre = np.nan
                substitute = re.compile("\s*(L|l)yrics.*")
                artist_name = re.sub(substitute, "", artist_name).replace("\n", "").replace("\t", "")
                artists[artist_name] = artist_url, artist_genre
                logging.info(f"Got {artist_name}")
            except AttributeError:
                logging.warning(f"Unable to retrieve other artist number {idx}. Html structure not aligned")        
    except AttributeError:
        logging.warning(f"Unable to retrieve other artists. Html structure not aligned")
    
    logging.info(f"Retrieved {len(artists.keys())} artists for letter {letter}")
    
    return artists
        
        
def retrieve_songs(artist, artist_url, driver, per_artist_limit):

    logging.info(f"\nRetrieving songs for {artist}")
    
    driver.get(artist_url)
    html = driver.page_source
    logging.info("Html source loaded")
    soup = BeautifulSoup(html, "lxml")
    
    try:
        titles_soup = soup.find("div", id="popular").find("div", class_="content").find("tbody").find_all("tr")
    except AttributeError:
        logging.warning(f"Unable to retrieve songs for {artist}. Html structure not aligned")
    
    titles = []
    if per_artist_limit is None:
        per_artist_limit = len(titles_soup)
    for idx, song in enumerate(titles_soup[:per_artist_limit]):
        try:
            substitute = re.compile("\s*(L|l)yrics.*")
            title = song.find("a", class_=["title hasvidtable", "title "]).get_text()
            title = re.sub(substitute, "", title).replace("\n", "").replace("\t", "")
            titles.append(title)
            logging.info(f"Got {title}")
        except AttributeError:
            logging.warning(f"Unable to retrieve song number {idx}. Html structure not aligned")
    
    logging.info(f"Retrieved {len(titles)} songs for {artist}")
    
    return titles

### Use Genius API to get lyrics 

In [22]:
def retrieve_lyrics(df_songs, genius_api, output_filename, avoid_artists=[]):
    
    dataset = df_songs
    artists = [arts for arts in dataset.artist.unique()[::-1] if arts not in avoid_artists]
    logging.info(f"Starting after {avoid_artists[-1]}")
    counter = 0
    counter_overall = 0
    
    for artist in artists:
        logging.info(f"\nRetrieving lyrics for {artist}")
        songs = dataset.loc[dataset.artist == artist, "song"].to_list()
        album = []
        lyrics = []
        release_date = []
        for idx, song_title in enumerate(songs):
            
            try:
                song = genius_api.search_song(song_title, artist=artist)
            except Exception as e:           
                try:
                    logging.warning(f"{song_title}, {artist} not found: {e.message} {e.args}")
                except AttributeError:
                    logging.warning(f"{song_title}, {artist} {song_title}, {artist} not found: {e}")
                
                dataset = dataset.drop(
                                index=dataset.loc[(dataset.artist == artist) & (dataset.song == song_title)].index
                                ).reset_index(drop=True)
                
                counter_overall += 1
                print(f"\rLyrics retrieved: {counter} Total songs: {counter_overall}", end='\r')
                sys.stdout.flush()
                continue
            
            try:
                text = song.lyrics
                substitute = re.compile("\[.+\]")
                text = re.sub(substitute, "", text).replace("\n", " ").replace("\t", " ")
                
                try:
                    language = TextBlob(text).detect_language()
                except Exception as e:
                    try:
                        logging.warning(f"Problem detecting language for {song_title}, {artist}: {e.message} {e.args}")
                    except AttributeError:
                        logging.warning(f"Problem detecting language for {song_title}, {artist}: {e}")
                    language = "en"
                if language != "en":
                    dataset = dataset.drop(
                                index=dataset.loc[(dataset.artist == artist) & (dataset.song == song_title)].index
                                ).reset_index(drop=True)
                    logging.warning(f"Excluded {song_title}, {artist}. Lyrics language: {language}")
                    counter_overall += 1
                    print(f"\rLyrics retrieved: {counter} Total songs: {counter_overall}", end='\r')
                    sys.stdout.flush()
                    continue
                else:
                    if len(text.split()) < 10000:
                        lyrics.append(text)
                    # avoid outliers --> surely not song lyrics
                    else:
                        dataset = dataset.drop(
                                index=dataset.loc[(dataset.artist == artist) & (dataset.song == song_title)].index
                                ).reset_index(drop=True)
                        logging.warning(f"Excluded {song_title}, {artist}. Lyrics longer than 10000 words")
                        counter_overall += 1
                        print(f"\rLyrics retrieved: {counter} Total songs: {counter_overall}", end='\r')
                        sys.stdout.flush()
                        continue
                        
            except AttributeError:
                logging.warning(f"No lyrics for {song_title}. Dropping")
                dataset = dataset.drop(
                                index=dataset.loc[(dataset.artist == artist) & (dataset.song == song_title)].index
                                ).reset_index(drop=True)
                counter_overall += 1
                print(f"\rLyrics retrieved: {counter} Total songs: {counter_overall}", end='\r')
                sys.stdout.flush()
                continue
            
            try:
                album.append(song.album.replace("\n", "").replace("\t", ""))
            except AttributeError:
                album.append(np.nan)
            
            try:
                release_date.append(song.year.replace("\n", "").replace("\t", ""))
            except AttributeError:
                release_date.append(np.nan)
            
            logging.info(f"Got data for {song_title}")
            counter += 1
            counter_overall += 1
            print(f"\rLyrics retrieved: {counter} Total songs: {counter_overall}", end='\r')
            sys.stdout.flush()
        
        logging.info(f"Retrieved lyrics for {artist}")
        logging.info(f"Artist Dataset shape: {dataset.loc[dataset.artist == artist, :].shape}, Albums: {len(album)}, Lyrics: {len(lyrics)}, Release Date: {len(release_date)}")
        
        dataset.loc[dataset.artist == artist, "album"] = album
        dataset.loc[dataset.artist == artist, "lyrics"] = lyrics
        dataset.loc[dataset.artist == artist, "release_date"] = release_date        
        
        logging.info(f"Writing {artist} to csv")
        logging.info(f"Global Dataset shape: {dataset.shape}")
        dataset.to_csv(f"/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/{output_filename}.csv", index=False)
    
    return dataset

### Final dataframe

In [5]:
df_initial = pd.DataFrame({"artist":[], "song":[], "album":[], "release_date":[], "genre":[], "lyrics":[]})
df_initial.head()

,artist,song,album,release_date,genre,lyrics


### Get songs

In [ ]:
driver = driver = webdriver.Firefox(executable_path="/Users/lucamasserano/Desktop/BOCCONI/Business Analytics/geckodriver")
songs = metrolyrics_retriever(df_initial, driver)
songs.shape

In [6]:
songs = pd.read_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/songs.csv")

In [7]:
songs.shape

(62470, 6)

### Get lyrics

In [9]:
api = lyricsgenius.Genius("BywKEeMHzO4cTLacaE-fUln9w7quBz13Hj6c10_Pm172KFstTUt_I3nJGFIPTxo9", sleep_time=0.01, verbose=False)

In [19]:
arts_1 = pd.read_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/first_part.csv").artist.unique()
arts_1

array(['6ix9ine', '50 Cent', '2Pac', '30 Seconds to Mars', '6cyclemind',
       '5 Seconds Of Summer', '4 Non Blondes', '69 Boyz', '3 Doors Down',
       "3 Lions '98", '52 Savage', '21 Savage', '911', '311', '88RISING',
       '2 Chainz', '38 Special', '10,000 Maniacs', '10 Years',
       '2 Live Crew', '98 Degrees', '10cc', '3OH!3', '112', '1000Mods',
       '6LACK', '1140 Mississippi', '11th Hour', '2 Unlimited', '4 Him',
       '2Face Idibia', '20 Fingers', '3 Piece', '30H3', '2NE1',
       '1st Lady', '702', '7horse', '36 Crazyfists', '047', '3T', '3LAU',
       'Atif Aslam', 'ABBA', 'Adele', 'A. R. Rahman', 'Annie',
       'Armaan Malik', 'Alicia Keys', 'Ariana Grande', 'Aladdin',
       'Amrit Maan', 'Avril Lavigne', 'Amy Winehouse', 'Aizat Amdan',
       'ARASH', 'Alan Walker', 'Akon', 'Andrea Bocelli', 'Arctic Monkeys',
       'ASAP Rocky', 'Aventura', 'Ammy Virk', 'Avicii',
       'Anthony Brown & Group Therapy', 'Adhitia Sofyan', 'Aerosmith',
       'Alan Jackson', 'Anna Ken

In [ ]:
arts_2 = pd.read_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/second_part.csv")
arts_2.artist.unique()

In [21]:
arts_retrieved = np.concatenate((arts_1, arts_2.loc[arts_2.lyrics.notnull(), :].artist.unique())) 
arts_retrieved

array(['6ix9ine', '50 Cent', '2Pac', '30 Seconds to Mars', '6cyclemind',
       '5 Seconds Of Summer', '4 Non Blondes', '69 Boyz', '3 Doors Down',
       "3 Lions '98", '52 Savage', '21 Savage', '911', '311', '88RISING',
       '2 Chainz', '38 Special', '10,000 Maniacs', '10 Years',
       '2 Live Crew', '98 Degrees', '10cc', '3OH!3', '112', '1000Mods',
       '6LACK', '1140 Mississippi', '11th Hour', '2 Unlimited', '4 Him',
       '2Face Idibia', '20 Fingers', '3 Piece', '30H3', '2NE1',
       '1st Lady', '702', '7horse', '36 Crazyfists', '047', '3T', '3LAU',
       'Atif Aslam', 'ABBA', 'Adele', 'A. R. Rahman', 'Annie',
       'Armaan Malik', 'Alicia Keys', 'Ariana Grande', 'Aladdin',
       'Amrit Maan', 'Avril Lavigne', 'Amy Winehouse', 'Aizat Amdan',
       'ARASH', 'Alan Walker', 'Akon', 'Andrea Bocelli', 'Arctic Monkeys',
       'ASAP Rocky', 'Aventura', 'Ammy Virk', 'Avicii',
       'Anthony Brown & Group Therapy', 'Adhitia Sofyan', 'Aerosmith',
       'Alan Jackson', 'Anna Ken

In [23]:
lyrics = retrieve_lyrics(songs, api, output_filename="reversed", avoid_artists=arts_retrieved)
lyrics.shape

Timeout raised and caught:l songs: 549
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 4855
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 4958
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 5142
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 5183
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 5226
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 8326
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 8612
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed 

ValueError: cannot copy sequence with size 147 to array axis with dimension 146

### Get lyrics if `retrieve_lyrics()` stopped at some point. Restart after last artist retrieved

In [25]:
checkpoint = pd.read_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/reversed.csv")
artists_checkpoint = checkpoint.loc[checkpoint.lyrics.notnull(), "artist"].unique()
artists_checkpoint

array(['Red Hot Chili Peppers', 'Radiohead', 'Rammstein', 'Rod Stewart',
       'Ray Stevens', 'Race', 'Rascal Flatts', 'Ray Charles',
       'Regine Velasquez', 'Rocky Horror Picture Show', 'Roy Orbison',
       'R. Kelly', 'Russ', 'Rodney Carrington', 'The Righteous Brothers',
       'Robbie Williams', 'Roan Ash', 'Reba McEntire',
       'Ricky Dillard & New G', 'Rush', 'Roger Miller', 'Reed Foehl',
       'Randy Newman', 'Rupert Holmes', 'Romeo Santos', 'Roberta Flack',
       'The Ramones', 'REM', 'Ricky Martin', 'Roxette', 'Raisa',
       'Ray LaMontagne', 'Robyn', 'Ram Jam', 'Sabyan Gambus',
       'SpongeBob SquarePants', 'Sound Of Music', 'Sugar free',
       'Sam Smith', 'Sia', 'Silent Sanctuary', 'Shawn Mendes',
       'Selena Gomez', 'South Park', 'Shakira', 'Sarah Geronimo',
       'Scooby Doo Theme Songs', 'Simon And Garfunkel', 'Spongecola',
       'Sesame Street', 'Stephen Bishop', 'Shankar Mahadevan',
       'Steven Universe', 'Songs For Children', 'Soundtrack',
       

In [55]:
lyrics = retrieve_lyrics(checkpoint, api, avoid_artists=artists_checkpoint)
lyrics.shape

Timeout raised and caught: songs: 55
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)


KeyboardInterrupt: 

### As dataset gets bigger the retrieval slows down --> Partition the dataset and then concatenate

In [72]:
retrieved = pd.read_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/lyrics.csv")

In [73]:
artists_retrieved = retrieved.loc[retrieved.lyrics.notnull(), "artist"].unique()
artists_retrieved

array(['6ix9ine', '50 Cent', '2Pac', '30 Seconds to Mars', '6cyclemind',
       '5 Seconds Of Summer', '4 Non Blondes', '69 Boyz', '3 Doors Down',
       "3 Lions '98", '52 Savage', '21 Savage', '911', '311', '88RISING',
       '2 Chainz', '38 Special', '10,000 Maniacs', '10 Years',
       '2 Live Crew', '98 Degrees', '10cc', '3OH!3', '112', '1000Mods',
       '6LACK', '1140 Mississippi', '11th Hour', '2 Unlimited', '4 Him',
       '2Face Idibia', '20 Fingers', '3 Piece', '30H3', '2NE1',
       '1st Lady', '702', '7horse', '36 Crazyfists', '047', '3T', '3LAU',
       'Atif Aslam', 'ABBA', 'Adele', 'A. R. Rahman', 'Annie',
       'Armaan Malik', 'Alicia Keys', 'Ariana Grande', 'Aladdin',
       'Amrit Maan', 'Avril Lavigne', 'Amy Winehouse', 'Aizat Amdan',
       'ARASH', 'Alan Walker', 'Akon', 'Andrea Bocelli', 'Arctic Monkeys',
       'ASAP Rocky', 'Aventura', 'Ammy Virk', 'Avicii',
       'Anthony Brown & Group Therapy', 'Adhitia Sofyan', 'Aerosmith',
       'Alan Jackson', 'Anna Ken

In [65]:
artists_left = [a for a in retrieved.artist.unique() if a not in artists_retrieved]
artists_left

['Eros Ramazzotti',
 'Edguy',
 'Eva Cassidy',
 'Empire Of The Sun',
 'Frank Sinatra',
 'Fleetwood Mac',
 'Farida Khanum',
 'Fiona Apple',
 'Foo Fighters',
 'Fall Out Boy',
 'Frank Ocean',
 'Freestyle',
 'Florence + The Machine',
 'Fritz Hagen',
 'Frank Zappa',
 'Fred Hammond',
 'Freddie Aguilar',
 'Florida Georgia Line',
 'Foster The People',
 'Fergie',
 'Flo Rida',
 'Fun.',
 'Future',
 'Flatt and Scruggs',
 'Farhan Saeed',
 'Fuzon',
 'Fifth Harmony',
 'Fats Domino',
 'Foreigner',
 'Flight of the Conchords',
 'The Fray',
 'Faith Hill',
 'Five Finger Death Punch',
 'Fabolous',
 'The Fugees',
 'Free Chapel',
 'Fever Ray',
 'Father Ray Kelly',
 'The Four Seasons',
 'Fenix',
 'Fiona Fung',
 'Five For Fighting',
 'Four Tops',
 'Fairport Convention',
 'Frankie Laine',
 'Franz Ferdinand',
 'Fatboy Slim',
 'Frazey Ford',
 'Faron Young',
 'Fela Kuti',
 'Guru Randhawa',
 'Grease',
 'Garth Brooks',
 'Gurnam Bhullar',
 'Green Day',
 'Gary Valenciano',
 'Grateful Dead',
 'Goo Goo Dolls',
 'Gaither 

In [66]:
first_part = retrieved.loc[retrieved.artist.isin(artists_retrieved), :]
first_part

,artist,song,album,release_date,genre,lyrics
0,6ix9ine,FEFE,DUMMY BOY,2018-07-22,NaN,"It's fuckin' TR3YWAY! King of New York, looki..."
1,6ix9ine,KEKE,DAY69: Graduation Day,2018-01-14,NaN,"Scum Gang! AM Nation I be on the block, on ..."
2,6ix9ine,TATI,DAY69: Graduation Day (Japanese Import),2018-05-27,NaN,"Hold up, let me get it started B.B. with the ..."
3,6ix9ine,BILLY,DAY69: Graduation Day,1417-06-12,NaN,"That's my word, get up in they face Talk your..."
4,6ix9ine,Gummo,DAY69: Graduation Day,2017-10-08,NaN,"Niggas iffy, uh, blicky got the stiffy, uh Go..."
5,6ix9ine,GOTTI,DAY69: Graduation Day,2018-04-10,NaN,Got me a new Rollie (You can't even afford th...
6,6ix9ine,Kooda,DAY69: Graduation Day,2017-12-03,NaN,"Scum Gang! Niggas runnin' out they mouth, b..."
7,6ix9ine,RONDO,DAY69: Graduation Day,2018-02-22,NaN,I'ma do my dance in this bih (I'ma do my danc...
8,6ix9ine,Blood Walk,NaN,2018-06-22,NaN,"Akademiks Music Suwoo! Gltt, brra, brra, brra..."
9,6ix9ine,GUMMO (Remix),DAY69: Graduation Day,2018-02-23,NaN,"Niggas iffy, uh, Blicky got the stiffy, uh Go..."


In [70]:
first_part.to_csv("/Users/lucamasserano/Desktop/BOCCONI/nlp/final_project/lyrics_project/first_part.csv", index=False)

In [69]:
second_part = retrieved.loc[~retrieved.artist.isin(artists_retrieved), :]
second_part

,artist,song,album,release_date,genre,lyrics
12453,Eros Ramazzotti,Fuoco Nel Fuoco,NaN,NaN,Pop,NaN
12454,Eros Ramazzotti,Parla Con Me,NaN,NaN,Pop,NaN
12455,Eros Ramazzotti,I Belong To You,NaN,NaN,Pop,NaN
12456,Eros Ramazzotti,L'aurora,NaN,NaN,Pop,NaN
12457,Eros Ramazzotti,Un' Altra Te,NaN,NaN,Pop,NaN
12458,Eros Ramazzotti,Pi Bella Cosa,NaN,NaN,Pop,NaN
12459,Eros Ramazzotti,Se Bastasse Una Canzone,NaN,NaN,Pop,NaN
12460,Eros Ramazzotti,Una Storia Importante,NaN,NaN,Pop,NaN
12461,Eros Ramazzotti,Non Siamo Soli,NaN,NaN,Pop,NaN
12462,Eros Ramazzotti,Terra Promessa,NaN,NaN,Pop,NaN


In [ ]:
lyrics = retrieve_lyrics(second_part, api, output_filename="second_part", avoid_artists=artists_retrieved)
lyrics.shape

Timeout raised and caught:al songs: 1168
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 2464
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 3503
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
Timeout raised and caught:al songs: 3633
HTTPSConnectionPool(host='api.genius.com', port=443): Read timed out. (read timeout=5)
